In [1]:
import os, sys

src_dir_ = '/home/tan/Documents/GitHub/pdpt_2022/src'
sys.path.insert(1, src_dir_)

import numpy as np
import matplotlib.pyplot as plt
import time
from gurobipy import Model, quicksum, GRB

from pdotw_mip import group_cycle_truck, postprocess_solution_pdotw, eval_pdotw_sol
from util import generate_node_cargo_size_change


In [2]:
def optimize_pdotw_route_gurobi(constant, y_sol,
    selected_cargo, single_truck_deviation,
    created_truck_yCycle, created_truck_nCycle, created_truck_all,
    node_list_truck_hubs, selected_edge, node_cargo_size_change, 
    runtime, filename, verbose = 0):
    
    """
    This model is a modified version of the PDOTW model.

    Now, y_sol, the cargo to truck assignment is given, so the goal is to minimize the traveling distance.

    return 
    1. obj_val_MP: objective value
    2. runtime_MP: runtime
    3. x_sol, _, _ , S_sol, D_sol, A_sol, 
       Sb_sol, Db_sol, Ab_sol: values of variables
        
    """
    
    MP = Model("Gurobi MIP for reoptimizing PDOTW routes")
    

    # binary variables y
    # ==1 if cargo_ is carried by truck_
    y = {}
    for truck_ in created_truck_all.keys():
        for cargo_ in selected_cargo.keys():
            y[(truck_, cargo_)] = y_sol[(truck_, cargo_)]

    ###### Decision Variables ######
    ###### six types of decision variables ######
    
    # binary variables x
    # ==1 if the truck_ traverse an edge 
    x = {}
    for truck_ in created_truck_all.keys():
        for i in node_list_truck_hubs[truck_]:
            for j in node_list_truck_hubs[truck_]:
                if i != j:
                    x[(i, j, truck_)] = \
                    MP.addVar(vtype=GRB.BINARY)
    


    # Integer variables S
    # current total size of cargos on truck_ at node_
    S = {}
    Sb = {}
    for truck_ in created_truck_all.keys():
        for node_ in node_list_truck_hubs[truck_]:
            S[(node_, truck_)] = MP.addVar(vtype=GRB.INTEGER, lb=0,
                                           ub=created_truck_all[truck_][3])
    # if truck_ is a cycle truck and node_ is its destination
    # add a decision variable Sb
    # NOT A D VARIABLE ANYMORE
    for truck_ in created_truck_yCycle.keys():
        node_ = created_truck_yCycle[truck_][1]
        Sb[(node_, truck_)] = 0
    
    # integer variable D
    # departure time of truck_ at node_
    D = {}
    Db = {}
    for truck_ in created_truck_all.keys():
        for node_ in node_list_truck_hubs[truck_]:
            D[(node_, truck_)] = MP.addVar(vtype=GRB.INTEGER, lb=0)
    # if truck_ is a cycle truck and node_ is its destination
    # add a decision variable Ab
    for truck_ in created_truck_yCycle.keys():
        node_ = created_truck_yCycle[truck_][1]
        Db[(node_, truck_)] = MP.addVar(vtype=GRB.INTEGER, lb=0)
    
    # integer variable A
    # arrival time of truck_ at node_
    A = {}
    Ab = {}
    for truck_ in created_truck_all.keys():
        for node_ in node_list_truck_hubs[truck_]:
            A[(node_, truck_)] = MP.addVar(vtype=GRB.INTEGER, lb=0)
    # if truck_ is a cycle truck and node_ is its destination
    # add a decision variable Db
    for truck_ in created_truck_yCycle.keys():
        node_ = created_truck_yCycle[truck_][1]
        Ab[(node_, truck_)] = MP.addVar(vtype=GRB.INTEGER, lb=0)
    
    ###### Constraints ######
    ###### Distinguish cycle trucks and non-cycle trucks ######
    
    # Flow constraints (3.1)
    # the truck must start from its origin
    for truck_ in created_truck_all.keys():
        origin_truck = created_truck_all[truck_][0]
        if origin_truck in node_list_truck_hubs[truck_]:
            MP.addConstr( 
                quicksum(x[(origin_truck, succ_node, truck_)] * 1
                         for succ_node in node_list_truck_hubs[truck_]
                         if succ_node != origin_truck) == 1 
            )
        
    # Flow constraints (3.2)  
    # only applies to non-cycle trucks
    # no flow enters the origin of a non-cycle truck
    for truck_ in created_truck_nCycle.keys():
        origin_truck = created_truck_nCycle[truck_][0]
        if origin_truck in node_list_truck_hubs[truck_]:
            MP.addConstr( 
                quicksum(x[(succ_node, origin_truck, truck_)] * 1
                         for succ_node in node_list_truck_hubs[truck_]
                         if succ_node != origin_truck) == 0 
            )

    # Flow constraints (3.3)
    # the truck must end at its destination
    for truck_ in created_truck_all.keys():
        destination_truck = created_truck_all[truck_][1]
        if destination_truck in node_list_truck_hubs[truck_]:
            MP.addConstr( 
                quicksum(x[(pred_node, destination_truck, truck_)] * 1
                         for pred_node in node_list_truck_hubs[truck_]
                         if pred_node != destination_truck) == 1
            )    
        
    # Flow constraints (3.4)
    # only applies to non-cycle trucks
    # no flow departs from the destination of a non-cycle truck
    for truck_ in created_truck_nCycle.keys():
        destination_truck = created_truck_nCycle[truck_][1]
        if destination_truck in node_list_truck_hubs[truck_]:
            MP.addConstr( 
                quicksum(x[(destination_truck, pred_node, truck_)] * 1
                         for pred_node in node_list_truck_hubs[truck_]
                         if pred_node != destination_truck) == 0 
            )
    
    
    ### No cycle part below ----------------------------------------------
    
    # Flow constraints (3.5)
    # flow in = flow out
    # Don't consider origin_truck and destination_truck in this constraint
    for truck_ in created_truck_all.keys():
        origin_truck = created_truck_all[truck_][0]
        destination_truck = created_truck_all[truck_][1]
        for node_ in node_list_truck_hubs[truck_]:
            if node_ != origin_truck and node_ != destination_truck:
                MP.addConstr(
                    quicksum(x[(pred_node, node_, truck_)] * 1
                             for pred_node in node_list_truck_hubs[truck_]
                             if pred_node != node_) 
                    ==
                    quicksum(x[(node_, succ_node, truck_)] * 1
                             for succ_node in node_list_truck_hubs[truck_]
                             if succ_node != node_) 
                )
    
    # An edge is used at most once by a truck (3.6)
    # only apply for non-cycle trucks
    # and non-origin nodes for cycle trucks
    for truck_ in created_truck_nCycle.keys():
        for i in node_list_truck_hubs[truck_]:
            for j in node_list_truck_hubs[truck_]:
                if i != j:
                    MP.addConstr(
                        x[(i, j, truck_)] +
                        x[(j, i, truck_)]
                        <= 1
                    )
    for truck_ in created_truck_yCycle.keys():
        origin_truck = created_truck_yCycle[truck_][0]
        for i in node_list_truck_hubs[truck_]:
            for j in node_list_truck_hubs[truck_]:
                if i != j:
                    if i != origin_truck and j != origin_truck:
                        MP.addConstr(
                            x[(i, j, truck_)] +
                            x[(j, i, truck_)]
                            <= 1
                        )
            
    
    # origin_c is visited by truck_ if y[(truck_, c)] == 1 (3.9)
    for truck_ in created_truck_all.keys():
        for cargo_ in selected_cargo.keys():
            origin_cargo = selected_cargo[cargo_][3]
            if origin_cargo in node_list_truck_hubs[truck_]:
                MP.addConstr(
                    quicksum(x[(origin_cargo, node_, truck_)] * 1
                             for node_ in node_list_truck_hubs[truck_]
                             if node_ != origin_cargo)
                    >= 
                    y[(truck_, cargo_)]
                )
    
    # destination_c is visited by truck_ if y[(truck_, c)] == 1 (3.10)
    for truck_ in created_truck_all.keys():
        for cargo_ in selected_cargo.keys():
            destination_cargo = selected_cargo[cargo_][4]
            if destination_cargo in node_list_truck_hubs[truck_]:
                MP.addConstr(
                    quicksum(x[(node_, destination_cargo, truck_)] * 1
                             for node_ in node_list_truck_hubs[truck_]
                             if node_ != destination_cargo)
                    >= 
                    y[(truck_, cargo_)]
                )
    
    ### Capacity ----------------------------------------------------
    
    # capacity constraints (3.14)
    for truck_ in created_truck_all.keys():
        # if truck_ is a NON-cycle truck and node_ is its destination
        # then the truck capacity when departing its destination is 0
        if truck_ in created_truck_nCycle.keys():
            destination_truck = created_truck_all[truck_][1]
            MP.addConstr(
                S[(destination_truck, truck_)] 
                == 0
            )
            
    # Cumulative total size of a truck at a node (3.15)
    # be aware of whether the node is 
    # a cargo origin or cargo destination, or both
    bigM_capacity = 30000
    for truck_ in created_truck_all.keys():
        for node1 in node_list_truck_hubs[truck_]:
            for node2 in node_list_truck_hubs[truck_]:
                if node1 != node2:
                    # if truck_ is a cycle truck 
                    # and node2 is its destination
                    if truck_ in created_truck_yCycle.keys():
                        if node2 == created_truck_yCycle[truck_][1]:
                            MP.addConstr(
                                Sb[(node2, truck_)] - S[(node1, truck_)]
                                >= 
                                quicksum(y[(truck_, cargo_)] * 
                                node_cargo_size_change[(node2, cargo_)]
                                for cargo_ in selected_cargo.keys()
                                if selected_cargo[cargo_][4] == node2)
                                - bigM_capacity 
                                * (1 - x[(node1, node2, truck_)])
                            )
                        else:
                            MP.addConstr(
                                S[(node2, truck_)] - S[(node1, truck_)]
                                >= 
                                quicksum(y[(truck_, cargo_)] * 
                                node_cargo_size_change[(node2, cargo_)]
                                for cargo_ in selected_cargo.keys()
                                if selected_cargo[cargo_][4] == node2 \
                                or selected_cargo[cargo_][3] == node2)
                                - bigM_capacity 
                                * (1 - x[(node1, node2, truck_)])
                            )
                    # else
                    else:
                        if node2 == created_truck_nCycle[truck_][1]:
                            MP.addConstr(
                                S[(node2, truck_)] - S[(node1, truck_)]
                                >= 
                                quicksum(y[(truck_, cargo_)] * 
                                node_cargo_size_change[(node2, cargo_)]
                                for cargo_ in selected_cargo.keys()
                                if selected_cargo[cargo_][4] == node2)
                                - bigM_capacity 
                                * (1 - x[(node1, node2, truck_)])
                            )
                        else:
                            MP.addConstr(
                                S[(node2, truck_)] - S[(node1, truck_)]
                                >= 
                                quicksum(y[(truck_, cargo_)] * 
                                node_cargo_size_change[(node2, cargo_)]
                                for cargo_ in selected_cargo.keys()
                                if selected_cargo[cargo_][4] == node2 \
                                or selected_cargo[cargo_][3] == node2)
                                - bigM_capacity 
                                * (1 - x[(node1, node2, truck_)])
                            )
    # Change 20220911 TAN
    # Add total size of cargo <= M * sum_j x^k(i,j)
    for truck_ in created_truck_all.keys():
        for node1 in node_list_truck_hubs[truck_]:
            # if truck_ is a cycle truck 
            # and node2 is its destination
            MP.addConstr(
                S[(node1, truck_)]<= 
                bigM_capacity * quicksum(x[(node1, node2, truck_)]\
                        for node2 in node_list_truck_hubs[truck_] if node1 != node2))

    # total size of cargos at truck origins (3.16)  
    # Should be an equality constraint
    for truck_ in created_truck_all.keys():
        origin_truck = created_truck_all[truck_][0]
        MP.addConstr(
            S[(origin_truck, truck_)] == 
            quicksum(y[(truck_, cargo_)] * \
                     node_cargo_size_change[(origin_truck, cargo_)]
                     for cargo_ in selected_cargo.keys()
                     if selected_cargo[cargo_][3] == origin_truck)
        )
    
    ### Time --------------------------------------------------------
    # The arrival time of a truck at any node (even if not visited) 
    # is less than or equal to the departure time of a truck
    for truck_ in created_truck_all.keys():
        for node_ in node_list_truck_hubs[truck_]:
            if truck_ in created_truck_yCycle.keys() and \
               node_ == created_truck_yCycle[truck_][1]:
                MP.addConstr(
                    Ab[(node_, truck_)] 
                    <= Db[(node_, truck_)]
                )
                MP.addConstr(
                    A[(node_, truck_)] 
                    <= D[(node_, truck_)]
                )
            else:
                MP.addConstr(
                    A[(node_, truck_)] 
                    <= D[(node_, truck_)]
                )
    
    # loading and unloading time between arrival and departure (3.17)
    # Don't consider origin_truck in this constraint
    # but for cycle trucks, their origins are also their destinations
    # so we only consider their destination parts
    for truck_ in created_truck_all.keys():
        for node_ in node_list_truck_hubs[truck_]:
            # if truck_ is a cycle truck
            if truck_ in created_truck_yCycle.keys():
                # if node_ is its destination
                if node_ == created_truck_yCycle[truck_][1]:
                    MP.addConstr(
                        Ab[(node_, truck_)] +
                        constant['node_fixed_time'] +
                        quicksum(y[(truck_, cargo_)] * 
                                 int(np.ceil(selected_cargo[cargo_][0] * 
                                 constant['loading_variation_coefficient']))
                                 for cargo_ in selected_cargo.keys()
                                 if node_ == selected_cargo[cargo_][4])
                        <= Db[(node_, truck_)]
                    )
                else:
                    MP.addConstr(
                        A[(node_, truck_)] +
                        constant['node_fixed_time'] +
                        quicksum(y[(truck_, cargo_)] * 
                                 int(np.ceil(selected_cargo[cargo_][0] * 
                                 constant['loading_variation_coefficient']))
                                 for cargo_ in selected_cargo.keys()
                                 if node_ == selected_cargo[cargo_][3] \
                                 or node_ == selected_cargo[cargo_][4])
                        <= D[(node_, truck_)]
                    )
            # if truck_ is a non-cycle truck
            else:
                if node_ != created_truck_all[truck_][0]:
                    if node_ == created_truck_all[truck_][1]:
                        MP.addConstr(
                            A[(node_, truck_)] +
                            constant['node_fixed_time'] +
                            quicksum(y[(truck_, cargo_)] * 
                                     int(np.ceil(selected_cargo[cargo_][0] * 
                                     constant['loading_variation_coefficient']))
                                     for cargo_ in selected_cargo.keys()
                                     if node_ == selected_cargo[cargo_][4])
                            <= D[(node_, truck_)]
                        )
                    else:
                        MP.addConstr(
                            A[(node_, truck_)] +
                            constant['node_fixed_time'] +
                            quicksum(y[(truck_, cargo_)] * 
                                     int(np.ceil(selected_cargo[cargo_][0] * 
                                     constant['loading_variation_coefficient']))
                                     for cargo_ in selected_cargo.keys()
                                     if node_ == selected_cargo[cargo_][3] \
                                     or node_ == selected_cargo[cargo_][4])
                            <= D[(node_, truck_)]
                        )
    
    # bigM constraints for travel time on edge(i,j) (3.18) 
    # D[prev_node] + edge[(prev_node, curr_node)] <= A[curr_node]
    bigM_time = 2000
    for truck_ in created_truck_all.keys():
        for node1 in node_list_truck_hubs[truck_]:
            for node2 in node_list_truck_hubs[truck_]:
                if node1 != node2:
                    # if truck_ is a cycle truck and 
                    # node2 is its destination
                    if truck_ in created_truck_yCycle.keys() and \
                       node2 == created_truck_yCycle[truck_][1]:
                        MP.addConstr(
                            D[(node1, truck_)] +
                            selected_edge[(node1, node2)]
                            <= 
                            Ab[(node2, truck_)] +
                            bigM_time * (1 - x[(node1, node2, truck_)])
                        )
                    else:
                        MP.addConstr(
                            D[(node1, truck_)] +
                            selected_edge[(node1, node2)]
                            <= 
                            A[(node2, truck_)] +
                            bigM_time * (1 - x[(node1, node2, truck_)])
                        )
    
    # Earliest time window of cargos (3.19)
    for truck_ in created_truck_all.keys():
        for cargo_ in selected_cargo.keys():
            origin_cargo = selected_cargo[cargo_][3]
            if origin_cargo in node_list_truck_hubs[truck_]:
                MP.addConstr(
                    D[(origin_cargo, truck_)]
                    >= 
                    selected_cargo[cargo_][1] * y[(truck_, cargo_)]
                )
            
    # Latest time window of cargos (3.20)
    for truck_ in created_truck_all.keys():
        for cargo_ in selected_cargo.keys():
            destination_cargo = selected_cargo[cargo_][4]
            if destination_cargo in node_list_truck_hubs[truck_]:
                # if truck_ is a cycle truck and 
                # destination_cargo is its destination
                if truck_ in created_truck_yCycle.keys() and \
                   destination_cargo == created_truck_yCycle[truck_][1]:
                    MP.addConstr(
                        Ab[(destination_cargo, truck_)]
                        <= 
                        selected_cargo[cargo_][2] + 
                        bigM_time * (1 - y[(truck_, cargo_)])
                    )
                else:
                    MP.addConstr(
                        A[(destination_cargo, truck_)]
                        <= 
                        selected_cargo[cargo_][2] + 
                        bigM_time * (1 - y[(truck_, cargo_)])
                    )

    # maximum worktime of trucks (3.21)
    for truck_ in created_truck_all.keys():
        origin_truck = created_truck_all[truck_][0]
        destination_truck = created_truck_all[truck_][1]
        # if truck_ is a cycle truck
        if truck_ in created_truck_yCycle.keys():
            MP.addConstr(
                Db[(destination_truck, truck_)] - D[(origin_truck, truck_)]
                <= 
                created_truck_yCycle[truck_][2]  # z[truck_] * 
            )
            MP.addConstr(
                Db[(destination_truck, truck_)] - D[(origin_truck, truck_)]
                >= 
                0  # z[truck_] * 
            )
        else:
            MP.addConstr(
                D[(destination_truck, truck_)] - D[(origin_truck, truck_)]
                <= 
                created_truck_all[truck_][2]  # z[truck_] * 
            )
            MP.addConstr(
                D[(destination_truck, truck_)] - D[(origin_truck, truck_)]
                >= 
                0  # z[truck_] * 
            )

    
    # first pickup and then delivery (3.22)
    for truck_ in created_truck_all.keys():
        for cargo_ in selected_cargo.keys():
            origin_cargo = selected_cargo[cargo_][3]
            destination_cargo = selected_cargo[cargo_][4]
            if destination_cargo in node_list_truck_hubs[truck_]:
                # if truck_ is a cycle truck and 
                # destination_cargo is its destination
                if truck_ in created_truck_yCycle.keys() and \
                   destination_cargo == created_truck_yCycle[truck_][1]:
                    MP.addConstr(
                        Ab[(destination_cargo, truck_)] - 
                        D[(origin_cargo, truck_)]
                        >= 
                        selected_edge[(origin_cargo, destination_cargo)] -
                        bigM_time * (1 - y[(truck_, cargo_)])
                    )
                    MP.addConstr(
                        A[(origin_cargo, truck_)] - 
                        D[(destination_cargo, truck_)]
                        >= 
                        selected_edge[(destination_cargo, origin_cargo)] -
                        bigM_time * (1 - y[(truck_, cargo_)])
                    )
                else:
                    MP.addConstr(
                        A[(destination_cargo, truck_)] - 
                        D[(origin_cargo, truck_)]
                        >= 
                        selected_edge[(origin_cargo, destination_cargo)] -
                        bigM_time * (1 - y[(truck_, cargo_)])
                    )
            
            
            
            
    
    ###### Objective ######
    
    # # traveling cost: proportional to total travel time
    cost_travel = quicksum(x[(node1, node2, truck_)] * 
                           selected_edge[(node1, node2)] * 
                           constant['truck_running_cost']
                           for truck_ in created_truck_all.keys()
                           for node1 in node_list_truck_hubs[truck_]
                           for node2 in node_list_truck_hubs[truck_]
                           if node1 != node2)

    # private parameters to help with callback function
    MP._cur_obj = float('inf')
    MP._time = time.time()
    MP._no_improve_iter = 0
    MP._termination_flag = 0

    MP.setObjective(cost_travel)
    MP.modelSense = GRB.MINIMIZE
    MP.Params.timeLimit = runtime
    MP.Params.OutputFlag = 1
    MP.Params.LogFile = filename
    MP.Params.LogToConsole  = 0
    MP.update()
    MP.optimize()


    # if infeasible
    if MP.Status == 3:
        if verbose >0: print('+++ MIP [Infeasible Proved] ')
        return -1, runtime, [], [], [], [], [], [], [], [], [], -1, -1, -1, -1
    
    runtime_MP = MP.Runtime
    obj_val_MP = MP.objVal

    
    # if no objective value
    if float('inf') == obj_val_MP:
        if verbose >0: print('+++ MIP [Infeasible] ')
        return -1, runtime, [], [], [], [], [], [], [], [], [], -1, -1, -1, -1
        
    
    if verbose > 0:
        print('+++ MP [Feasible] ')
        if MP._termination_flag == 1:
            print('soft termination: failed to improve best solution for 20s.')
        elif MP._termination_flag == 2:
            print('soft termination: failed to improve obj for 50 consecutive feasible solutions.')
        print("   [Gurobi obj value] is %i" % obj_val_MP)
        print("   [Gurobi runtime] is %f" % runtime_MP)
    
    
    
    ###### Get solutions ######
    
    # store all values in a list: sol
    sol = []
    for ss in MP.getVars():
        sol.append(int(ss.x))
        
    # retrieve values from the list sol
    count = 0
    # binary variables x
    x_sol = {}
    for truck_ in created_truck_all.keys():
        for i in node_list_truck_hubs[truck_]:
            for j in node_list_truck_hubs[truck_]:
                if i != j:
                    x_sol[(i, j, truck_)] = sol[count]
                    count += 1
                
    # # binary variables y
    # y_sol = {}
    # for truck_ in created_truck_all.keys():
    #     for cargo_ in selected_cargo.keys():
    #         y_sol[(truck_, cargo_)] = sol[count]
    #         count += 1
    
    # integer variable S
    S_sol = {}
    Sb_sol = {}
    for truck_ in created_truck_all.keys():
        for node_ in node_list_truck_hubs[truck_]:
            S_sol[(node_, truck_)] = sol[count]
            count += 1
    # if truck_ is a cycle truck and node_ is its destination
    for truck_ in created_truck_yCycle.keys():
        node_ = created_truck_yCycle[truck_][1]
        Sb_sol[(node_, truck_)] = 0
            
    # integer variable D
    D_sol = {}
    Db_sol = {}
    for truck_ in created_truck_all.keys():
        for node_ in node_list_truck_hubs[truck_]:
            D_sol[(node_, truck_)] = sol[count]
            count += 1
    # if truck_ is a cycle truck and node_ is its destination
    for truck_ in created_truck_yCycle.keys():
        node_ = created_truck_yCycle[truck_][1]
        Db_sol[(node_, truck_)] = sol[count]
        count += 1
    
    # integer variable A
    A_sol = {}
    Ab_sol = {}
    for truck_ in created_truck_all.keys():
        for node_ in node_list_truck_hubs[truck_]:
            A_sol[(node_, truck_)] = sol[count]
            count += 1
    # if truck_ is a cycle truck and node_ is its destination
    for truck_ in created_truck_yCycle.keys():
        node_ = created_truck_yCycle[truck_][1]
        Ab_sol[(node_, truck_)] = sol[count]
        count += 1
    
    if verbose > 1:
        print('+++ The x_sol:')
        for key, value in x_sol.items():
            if value == 1:
                print(f'        {key, value}')
        print('+++ The y_sol:')
        for key, value in y_sol.items():
            if value == 1:
                print(f'        {key, value}')

    # cargo cost: proportional to the total size of cargo carried by the only truck
    cost_cargo_size_value = 0
    for truck_ in created_truck_all.keys():
        for cargo_ in selected_cargo.keys():
            cost_cargo_size_value += \
            y_sol[truck_, cargo_] * selected_cargo[cargo_][0] * \
            constant['truck_running_cost'] * 1
    if verbose >1:
        print('+++ [cost_cargo_size_value] ', cost_cargo_size_value)
    
    # cargo number cost: proportional to the number of cargo carried by the only truck
    cost_cargo_number_value = 0
    for truck_ in created_truck_all.keys():
        for cargo_ in selected_cargo.keys():
            cost_cargo_number_value += \
            y_sol[truck_, cargo_] * constant['truck_running_cost'] * 1000
    if verbose >1:
        print('+++ [cost_cargo_number_value] ', cost_cargo_number_value)
    
    # traveling cost: proportional to total travel time
    cost_travel_value = 0
    for truck_ in created_truck_all.keys():
        for node1 in node_list_truck_hubs[truck_]:
            for node2 in node_list_truck_hubs[truck_]:
                if node1 != node2:
                    cost_travel_value += x_sol[(node1, node2, truck_)] * \
                    selected_edge[(node1, node2)] * \
                    constant['truck_running_cost']
    if verbose >1:
        print('+++ [cost_travel_value] ', cost_travel_value)
    
    # deviation cost: proportional to total deviation of cargo carried by the only truck
    cost_deviation_value = 0
    for truck_ in created_truck_all.keys():
        for cargo_ in selected_cargo.keys():
            cost_deviation_value += \
            y_sol[truck_, cargo_] * 1 * \
            single_truck_deviation[(cargo_, truck_)] * \
            constant['truck_running_cost']
    if verbose >1:
        print('+++ [cost_deviation_value] ', cost_deviation_value, '\n')
    
    
         
    del MP

    
    return obj_val_MP, runtime_MP, \
           x_sol, {}, {}, S_sol, D_sol, A_sol, \
           Sb_sol, Db_sol, Ab_sol, \
           cost_cargo_size_value, cost_cargo_number_value, \
           cost_travel_value, cost_deviation_value

In [3]:
from pandas import read_pickle

def optimize_pdotw_mip(ins,  # dict contains the data of pdpt instance,
                        path_, # file where all data of pdotw solutions are saved
                        verbose = 0):  





    # load data from ins
    truck_list = ins['truck']
    cargo_list = ins['cargo']
    selected_cargo = cargo_list.copy()

    # edges = ins['edges']
    # nodes = ins['nodes']
    constant = ins['constant']
    node_cargo_size_change = ins['node_cargo_size_change']
    edge_shortest = ins['edge_shortest']
    # path_shortest = ins['path_shortest']
    single_truck_deviation = ins['single_truck_deviation']

    # pdotw_sol = read_pickle(os.path.join(path_, 'toy_initSol.pkl'))
    # pdotw_sol = read_pickle(os.path.join(path_, 'toy_initSol.pkl'))
    pdotw_sol = read_pickle(path_+'_initSol.pkl')


    # res = {'MIP': {'x_sol': x_sol_total,
    #                'y_sol': y_sol_total,
    #                'S_sol': S_sol_total,
    #                'D_sol': D_sol_total,
    #                'A_sol': A_sol_total,
    #                'Sb_sol': Sb_sol_total,
    #                'Db_sol': Db_sol_total,
    #                'Ab_sol': Ab_sol_total,
    #                'runtime': runtime_pdotw,
    #               },
    #        'route': {'truck_yCycle':list(created_truck_yCycle_total.keys()),
    #                  'used_truck': truck_used_total,
    #                  'truck_route': truck_route,
    #                  'cargo_route': cargo_route,
    #                 },
    #        'cost': {'truck_cost' : truck_cost,
    #                 'travel_cost' : travel_cost,
    #                 },
    #       }


    y_sol = pdotw_sol['MIP']['y_sol']
    truck_route_sol = pdotw_sol['route']['truck_route']
    cargo_route_sol = pdotw_sol['route']['cargo_route']
    used_truck = pdotw_sol['route']['used_truck']

    carog_truck_assignment = {}

    truck_used_total = []
    truck_route = {}
    for t_key, t_value in truck_list.items():
        truck_route[t_key] = []
        truck_route[t_key].append(t_value[0])

    cargo_delivered_total = {} 
    cargo_undelivered_total = {} 
    cargo_route = {}
    truck_per_cargo = {}
    for c_key, _ in cargo_list.items():
        truck_per_cargo[c_key] = -1
        cargo_route[c_key] = []
    cargo_in_truck = {}

    for truck_key in used_truck:
        print(f' +++ START re-optimize PDOTW solution for  truck [{truck_key}] \n')
        carog_truck_assignment[truck_key] = [key[1] for key in y_sol.keys() if (key[0]==truck_key and y_sol[key]==1)]
        print(f' +++ Truck [{truck_key}]: [{truck_route_sol[truck_key]}]')
        print(f' +++ Cargos to deliver: [{carog_truck_assignment[truck_key]}]')

        tv = truck_list[truck_key]
        created_truck = {}
        created_truck[truck_key] = tv

        node_list_truck_hubs = {}
        delivered_cargo = {cargo_key: cargo_list[cargo_key] for cargo_key in carog_truck_assignment[truck_key]}
        
        # nodes in the cluster
        # Note. cargo['nb_cargo'] = ['size', 'lb_time', 'ub_time','departure_node', 'arrival_node']
        # truck['nb_truck'] = ['departure_node', 'arrival_node', 'max_worktime', 'max_capacity']

        selected_node = []
        for v in delivered_cargo.values():
            if v[3] not in selected_node:
                selected_node.append(v[3])
            if v[4] not in selected_node:
                selected_node.append(v[4])
        if tv[0] not in selected_node:
            selected_node.append(tv[0])
        if tv[1] not in selected_node:
            selected_node.append(tv[1])

    # edges in the cluster
        selected_edge = {}
        for i in selected_node:
            for j in selected_node:
                selected_edge[(i,j)] = edge_shortest[(i,j)]
        
        node_list_truck_hubs[truck_key] = selected_node.copy()
        assert len(created_truck) == len(node_list_truck_hubs), "Inconsistent truck numbers"
        node_cargo_size_change = \
        generate_node_cargo_size_change(selected_node, delivered_cargo)

        ### group cycle and non-cycle trucks
        created_truck_yCycle, created_truck_nCycle, created_truck_all = \
        group_cycle_truck(created_truck) 


        # gurobi_log_file = os.path.join(path_, f'toy_gurobi/truck{truck_key}_reoptimized.log')
        gurobi_log_file = path_+f'_gurobi/truck{truck_key}_reoptimized.log'

        obj_val_MP, runtime_MP, \
        x_sol, _, _, S_sol, D_sol, A_sol, \
        Sb_sol, Db_sol, Ab_sol, \
        cost_cargo_size_value, cost_cargo_number_value, \
        cost_travel_value, cost_deviation_value\
        = optimize_pdotw_route_gurobi(constant, y_sol,
        delivered_cargo, single_truck_deviation,
        created_truck_yCycle, created_truck_nCycle, created_truck_all,
        node_list_truck_hubs, selected_edge, node_cargo_size_change,
        100, gurobi_log_file, verbose = 1)

        x_sol_total, y_sol_total, S_sol_total,\
        D_sol_total, A_sol_total, Sb_sol_total,\
        Db_sol_total, Ab_sol_total = {}, {}, {}, {}, {}, {}, {}, {}

        if obj_val_MP >= 0:
            if verbose >0:
                print(f'+++ Postprocee Gurobi solution if a feasible solution is found')

            for key, value in x_sol.items():
                x_sol_total[key] = value
            for key, value in y_sol.items():
                y_sol_total[key] = value
            for key, value in S_sol.items():
                S_sol_total[key] = value
            for key, value in D_sol.items():
                D_sol_total[key] = value
            for key, value in A_sol.items():
                A_sol_total[key] = value
            for key, value in Sb_sol.items():
                Sb_sol_total[key] = value
            for key, value in Db_sol.items():
                Db_sol_total[key] = value
            for key, value in Ab_sol.items():
                Ab_sol_total[key] = value

            ### post-process the solution
            truck_used, cargo_delivered, cargo_undelivered, \
            cargo_truck_total, cargo_in_truck = \
            postprocess_solution_pdotw(cargo_list, truck_list, 
            delivered_cargo, created_truck_all,
            node_list_truck_hubs, 
            x_sol, y_sol, truck_route, cargo_route, verbose = verbose-1)

            print(f' +++ New route {truck_route[truck_key]}')

            for truck_ in truck_used:
                if truck_ not in truck_used_total:
                    truck_used_total.append(truck_)

            for c_key, c_value in cargo_delivered.items():
                cargo_delivered_total[c_key] = c_value

            for c_key, c_value in cargo_undelivered.items():
                cargo_undelivered_total[c_key] = c_value

            for c_key, c_value in cargo_truck_total.items():
                if v != -1:
                    truck_per_cargo[c_key] = c_value
            for t_key, t_value in cargo_in_truck.items():
                cargo_in_truck[t_key] = t_value.copy()

        print(f' +++ END re-optimize PDOTW solution for  truck [{truck_key}] \n')

    truck_cost, travel_cost = eval_pdotw_sol(constant, edge_shortest, truck_used_total, truck_route)
    old_cost = pdotw_sol['cost']['travel_cost']
    print(f' +++ old travel cost: [{old_cost}]')
    print(f' +++ new travel cost: [{travel_cost}]')


In [4]:
toy_path_ =  '/home/tan/Documents/GitHub/pdpt_2022/toy/'
fujitsu_path_ =  '/home/tan/Documents/GitHub/pdpt_2022/'

# pdpt_ins = read_pickle(os.path.join(path_, 'toy.pkl'))

# case_num=1

for case_num in range(1,6,1):
    print(f'====== Re-optimize PDOTW solutions for Case {case_num}')
    pdpt_ins = read_pickle(os.path.join(fujitsu_path_, 'data', f'case{case_num}.pkl'))
    path_ = fujitsu_path_ + f'out/case{case_num}'
    optimize_pdotw_mip(pdpt_ins, path_)

====== Re-optimize PDOTW solutions for Case 1
========= Reoptimize the route of PDOTW sol on Truck [T11]
 +++ Truck [T11]: [['N10', 'N19', 'N21', 'N14']]
 +++ Cargos to deliver: [['C9', 'C11', 'C12', 'C20', 'C29', 'C30', 'C31', 'C32', 'C88', 'C120', 'C139', 'C140', 'C145', 'C147', 'C152', 'C156', 'C170', 'C222', 'C232', 'C235']]
Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-15
Set parameter TimeLimit to value 100
Set parameter LogFile to value "/home/tan/Documents/GitHub/pdpt_2022/out/case1_gurobi/truckT11_reoptimized.log"
+++ MP [Feasible] 
   [Gurobi obj value] is 16950
   [Gurobi runtime] is 0.006580
 +++ New route ['N10', 'N19', 'N21', 'N14']
 +++ old travel cost: [552350.0]
 +++ new travel cost: [16950.0]
 +++ END re-optimize PDOTW solution for  truck [T11] 

========= Reoptimize the route of PDOTW sol on Truck [T20]
 +++ Truck [T20]: [['N11', 'N5', 'N20', 'N14', 'N1', 'N15']]
 +++ Cargos to deliver: [['C15', 'C55', 'C60', 'C63', 'C65', 'C